In [512]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

<b>SIGUE:</b> " Elegir al menos tres clases de la carpeta “Para la T5” (en la raíz del drive de la clase).<br> Elegir al menos 30 ejemplos para cada clase para entrenar. " 

<b>" Elegir al menos tres clases de la carpeta “Para la T5” " </b>

In [513]:
selected_classes = ['blurry-nothing', 'esophagitis', 'polyps']

leyendo los datos

In [514]:
# Step 2: Read Data and Prepare Features
data = []
labels = []

i = 0
for class_name in selected_classes: #iteramos sobre las carpetas/clases seleccionadas 
    examples_path = f'datasets/para_knn_t5/{class_name}/'
    examples = os.listdir(examples_path)
    
    #iteramos sobre cada archivo(ejemplo) de cada clase
    for example_file in examples:
        i += 1
        #file_path = f'datasets/para_knn_t5/{class_name}/{i}.features' #ruta de cada ejemplo
        file_path = f'datasets/para_knn_t5/{class_name}/{example_file}' #ruta de cada ejemplo
        
        with open(file_path, 'r') as file: #lectura de cada ejemplo
            content = file.read() #leemos el contenido del archivo

            lines = content.split('\n') #separamos el contenido por lineas
            
            # inicializamos un diccionario vacio para cada ejemplo (dict comprehension)
            features_dict = {label: [] for label in ["JCD", "Tamura", "ColorLayout", "EdgeHistogram", "AutoColorCorrelogram", "PHOG"]}
            
            for line in lines: #iteramos sobre cada linea del ejemplo
                
                parts = line.split(':') #separamos cada linea por el caracter ':'

                if len(parts) >= 2: #si hay al menos dos partes (label y valores)
                    label = parts[0].strip() #obtenemos el label y lo guardamos en la variable label
                    values = parts[1].strip().split(',') #obtenemos los valores y los guardamos en la variable values
                    #values = parts[1].split(',') #ES UNA LISTA DE STRINGS
                    # convertimos los valores a float y los agregamos a su respectivo diccionario
                    features_dict[label] = [float(value) for value in values]
                    #features_dict[label] = values
                    
                    
            # agregamos los diccionarios de cada ejemplo a la lista de datos y la clase a la lista de labels
            data.append(features_dict)
            labels.append(class_name)

print(i)
#representamos los datos en un dataframe con labels al principio
df = pd.DataFrame(data, index=labels)

#df.head()


1233


In [515]:
df.head(3)

,JCD,Tamura,ColorLayout,EdgeHistogram,AutoColorCorrelogram,PHOG
blurry-nothing,"[0.0, 0.0, 1.0, 0.0, 2.5, 0.0, 0.0, 7.0, 0.0, ...","[3.738020833333333, 1.7487498603102698, 368.0,...","[23.0, 14.0, 11.0, 10.0, 15.0, 18.0, 16.0, 16....","[1.0, 2.0, 2.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[10.0, 8.0, 7.0, 6.0, 1.0, 0.0, 0.0, 0.0, 0.0,...","[4.0, 2.0, 1.0, 1.0, 1.0, 2.0, 5.0, 9.0, 10.0,..."
blurry-nothing,"[0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.5, 7.0, 0.0, ...","[3.6579545454545452, 2.9437503120078268, 313.0...","[29.0, 14.0, 16.0, 8.0, 14.0, 18.0, 14.0, 16.0...","[1.0, 0.0, 7.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, ...","[13.0, 11.0, 11.0, 10.0, 8.0, 4.0, 2.0, 2.0, 0...","[12.0, 9.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 4.0,..."
blurry-nothing,"[0.0, 0.0, 2.5, 0.0, 0.0, 0.0, 0.5, 6.5, 6.5, ...","[3.6724537037037037, 1.7752175824926766, 385.0...","[11.0, 24.0, 9.0, 13.0, 14.0, 15.0, 18.0, 14.0...","[0.0, 0.0, 2.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[14.0, 13.0, 12.0, 12.0, 10.0, 5.0, 3.0, 3.0, ...","[5.0, 5.0, 1.0, 0.0, 0.0, 1.0, 3.0, 3.0, 0.0, ..."


In [516]:
#make index a column (class)
df.reset_index(level=0, inplace=True)
#make index a column (examples)
df.reset_index(level=0, inplace=True)
#change names of first three columns
df.rename(columns={"index": "class", "level_0": "example"}, inplace=True)

df

,example,class,JCD,Tamura,ColorLayout,EdgeHistogram,AutoColorCorrelogram,PHOG
0,0,blurry-nothing,"[0.0, 0.0, 1.0, 0.0, 2.5, 0.0, 0.0, 7.0, 0.0, ...","[3.738020833333333, 1.7487498603102698, 368.0,...","[23.0, 14.0, 11.0, 10.0, 15.0, 18.0, 16.0, 16....","[1.0, 2.0, 2.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[10.0, 8.0, 7.0, 6.0, 1.0, 0.0, 0.0, 0.0, 0.0,...","[4.0, 2.0, 1.0, 1.0, 1.0, 2.0, 5.0, 9.0, 10.0,..."
1,1,blurry-nothing,"[0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.5, 7.0, 0.0, ...","[3.6579545454545452, 2.9437503120078268, 313.0...","[29.0, 14.0, 16.0, 8.0, 14.0, 18.0, 14.0, 16.0...","[1.0, 0.0, 7.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, ...","[13.0, 11.0, 11.0, 10.0, 8.0, 4.0, 2.0, 2.0, 0...","[12.0, 9.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 4.0,..."
2,2,blurry-nothing,"[0.0, 0.0, 2.5, 0.0, 0.0, 0.0, 0.5, 6.5, 6.5, ...","[3.6724537037037037, 1.7752175824926766, 385.0...","[11.0, 24.0, 9.0, 13.0, 14.0, 15.0, 18.0, 14.0...","[0.0, 0.0, 2.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[14.0, 13.0, 12.0, 12.0, 10.0, 5.0, 3.0, 3.0, ...","[5.0, 5.0, 1.0, 0.0, 0.0, 1.0, 3.0, 3.0, 0.0, ..."
3,3,blurry-nothing,"[0.0, 0.0, 2.0, 0.0, 4.0, 0.0, 1.5, 7.0, 0.5, ...","[3.6527777777777777, 1.8756582269004431, 255.0...","[28.0, 14.0, 18.0, 13.0, 10.0, 11.0, 16.0, 14....","[0.0, 0.0, 2.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[13.0, 12.0, 11.0, 11.0, 8.0, 4.0, 2.0, 2.0, 0...","[13.0, 11.0, 2.0, 2.0, 2.0, 2.0, 2.0, 3.0, 4.0..."
4,4,blurry-nothing,"[0.0, 0.0, 2.0, 0.0, 3.0, 0.0, 4.0, 7.0, 0.0, ...","[3.6948529411764706, 3.6810063589253157, 419.0...","[34.0, 25.0, 17.0, 12.0, 18.0, 10.0, 15.0, 16....","[0.0, 0.0, 2.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, ...","[15.0, 15.0, 15.0, 14.0, 1.0, 0.0, 0.0, 0.0, 1...","[1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 2.0, 15.0,..."
...,...,...,...,...,...,...,...,...
1228,1228,polyps,"[0.0, 0.0, 2.0, 0.0, 0.0, 0.5, 4.5, 6.0, 5.0, ...","[3.565393518518518, 3.9398713446452707, 358.0,...","[21.0, 26.0, 16.0, 7.0, 16.0, 10.0, 15.0, 14.0...","[3.0, 1.0, 6.0, 1.0, 3.0, 5.0, 3.0, 6.0, 6.0, ...","[13.0, 12.0, 12.0, 11.0, 10.0, 7.0, 5.0, 4.0, ...","[4.0, 3.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
1229,1229,polyps,"[2.0, 0.0, 4.0, 0.0, 0.0, 0.0, 4.5, 3.5, 5.0, ...","[3.670138888888889, 7.001793726640179, 273.0, ...","[20.0, 22.0, 3.0, 8.0, 13.0, 6.0, 20.0, 24.0, ...","[1.0, 0.0, 6.0, 1.0, 4.0, 1.0, 2.0, 6.0, 1.0, ...","[14.0, 13.0, 13.0, 12.0, 14.0, 13.0, 13.0, 12....","[11.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0,..."
1230,1230,polyps,"[0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 4.5, 6.0, 3.5, ...","[3.6107954545454546, 4.083707645485576, 304.0,...","[27.0, 20.0, 24.0, 9.0, 7.0, 7.0, 15.0, 14.0, ...","[3.0, 1.0, 7.0, 0.0, 2.0, 1.0, 1.0, 2.0, 2.0, ...","[13.0, 11.0, 11.0, 10.0, 9.0, 6.0, 4.0, 3.0, 6...","[14.0, 12.0, 7.0, 7.0, 6.0, 5.0, 5.0, 5.0, 5.0..."
1231,1231,polyps,"[1.0, 0.0, 2.0, 0.0, 0.5, 0.5, 4.0, 5.5, 4.0, ...","[3.6423611111111107, 4.732241092609992, 319.0,...","[24.0, 24.0, 26.0, 12.0, 16.0, 11.0, 10.0, 10....","[2.0, 1.0, 7.0, 2.0, 4.0, 2.0, 4.0, 6.0, 4.0, ...","[12.0, 10.0, 9.0, 8.0, 13.0, 11.0, 10.0, 9.0, ...","[7.0, 5.0, 4.0, 3.0, 3.0, 4.0, 5.0, 5.0, 6.0, ..."


divide color layout list elements in its own row, mantaining the class and example

In [517]:
#divide color layout list elements in its own row, mantaining the class and example
#df = df.explode('colorLayout')
#df.head()

In [518]:
df.dtypes

example                  int64
class                   object
JCD                     object
Tamura                  object
ColorLayout             object
EdgeHistogram           object
AutoColorCorrelogram    object
PHOG                    object
dtype: object

<b>OBJECT!? </b> <br>
Lo solucionaremos mas adelante

only take color layout

In [519]:
#only take example, class and colorLayout
selected_feature = 'ColorLayout'
df = df[['example', 'class', selected_feature]]

df

,example,class,ColorLayout
0,0,blurry-nothing,"[23.0, 14.0, 11.0, 10.0, 15.0, 18.0, 16.0, 16...."
1,1,blurry-nothing,"[29.0, 14.0, 16.0, 8.0, 14.0, 18.0, 14.0, 16.0..."
2,2,blurry-nothing,"[11.0, 24.0, 9.0, 13.0, 14.0, 15.0, 18.0, 14.0..."
3,3,blurry-nothing,"[28.0, 14.0, 18.0, 13.0, 10.0, 11.0, 16.0, 14...."
4,4,blurry-nothing,"[34.0, 25.0, 17.0, 12.0, 18.0, 10.0, 15.0, 16...."
...,...,...,...
1228,1228,polyps,"[21.0, 26.0, 16.0, 7.0, 16.0, 10.0, 15.0, 14.0..."
1229,1229,polyps,"[20.0, 22.0, 3.0, 8.0, 13.0, 6.0, 20.0, 24.0, ..."
1230,1230,polyps,"[27.0, 20.0, 24.0, 9.0, 7.0, 7.0, 15.0, 14.0, ..."
1231,1231,polyps,"[24.0, 24.0, 26.0, 12.0, 16.0, 11.0, 10.0, 10...."


In [520]:
df.dtypes

example         int64
class          object
ColorLayout    object
dtype: object

In [521]:
# Convert 'ColorLayout' column to individual columns
df_colorlayout = pd.DataFrame(df['ColorLayout'].tolist(), columns=[f'ColorLayout_{i+1}' for i in range(len(df['ColorLayout'].iloc[0]))])

# Concatenate the new columns to the original DataFrame
df = pd.concat([df, df_colorlayout], axis=1)

# Drop the original 'ColorLayout' column
df = df.drop('ColorLayout', axis=1)

# Display the DataFrame
df.head(3)

,example,class,ColorLayout_1,ColorLayout_2,ColorLayout_3,ColorLayout_4,ColorLayout_5,ColorLayout_6,ColorLayout_7,ColorLayout_8,...,ColorLayout_24,ColorLayout_25,ColorLayout_26,ColorLayout_27,ColorLayout_28,ColorLayout_29,ColorLayout_30,ColorLayout_31,ColorLayout_32,ColorLayout_33
0,0,blurry-nothing,23.0,14.0,11.0,10.0,15.0,18.0,16.0,16.0,...,21.0,21.0,17.0,15.0,63.0,16.0,16.0,10.0,14.0,7.0
1,1,blurry-nothing,29.0,14.0,16.0,8.0,14.0,18.0,14.0,16.0,...,8.0,23.0,9.0,21.0,59.0,5.0,25.0,8.0,27.0,2.0
2,2,blurry-nothing,11.0,24.0,9.0,13.0,14.0,15.0,18.0,14.0,...,16.0,20.0,10.0,20.0,51.0,11.0,21.0,7.0,27.0,6.0


In [522]:
df.dtypes

example             int64
class              object
ColorLayout_1     float64
ColorLayout_2     float64
ColorLayout_3     float64
ColorLayout_4     float64
ColorLayout_5     float64
ColorLayout_6     float64
ColorLayout_7     float64
ColorLayout_8     float64
ColorLayout_9     float64
ColorLayout_10    float64
ColorLayout_11    float64
ColorLayout_12    float64
ColorLayout_13    float64
ColorLayout_14    float64
ColorLayout_15    float64
ColorLayout_16    float64
ColorLayout_17    float64
ColorLayout_18    float64
ColorLayout_19    float64
ColorLayout_20    float64
ColorLayout_21    float64
ColorLayout_22    float64
ColorLayout_23    float64
ColorLayout_24    float64
ColorLayout_25    float64
ColorLayout_26    float64
ColorLayout_27    float64
ColorLayout_28    float64
ColorLayout_29    float64
ColorLayout_30    float64
ColorLayout_31    float64
ColorLayout_32    float64
ColorLayout_33    float64
dtype: object

In [523]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Convert class labels to numerical labels
df['class_label'] = label_encoder.fit_transform(df['class'])

In [524]:
df

,example,class,ColorLayout_1,ColorLayout_2,ColorLayout_3,ColorLayout_4,ColorLayout_5,ColorLayout_6,ColorLayout_7,ColorLayout_8,...,ColorLayout_25,ColorLayout_26,ColorLayout_27,ColorLayout_28,ColorLayout_29,ColorLayout_30,ColorLayout_31,ColorLayout_32,ColorLayout_33,class_label
0,0,blurry-nothing,23.0,14.0,11.0,10.0,15.0,18.0,16.0,16.0,...,21.0,17.0,15.0,63.0,16.0,16.0,10.0,14.0,7.0,0
1,1,blurry-nothing,29.0,14.0,16.0,8.0,14.0,18.0,14.0,16.0,...,23.0,9.0,21.0,59.0,5.0,25.0,8.0,27.0,2.0,0
2,2,blurry-nothing,11.0,24.0,9.0,13.0,14.0,15.0,18.0,14.0,...,20.0,10.0,20.0,51.0,11.0,21.0,7.0,27.0,6.0,0
3,3,blurry-nothing,28.0,14.0,18.0,13.0,10.0,11.0,16.0,14.0,...,21.0,10.0,22.0,61.0,5.0,25.0,4.0,29.0,4.0,0
4,4,blurry-nothing,34.0,25.0,17.0,12.0,18.0,10.0,15.0,16.0,...,18.0,15.0,20.0,59.0,8.0,13.0,11.0,13.0,9.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1228,1228,polyps,21.0,26.0,16.0,7.0,16.0,10.0,15.0,14.0,...,21.0,12.0,21.0,50.0,8.0,24.0,6.0,26.0,5.0,2
1229,1229,polyps,20.0,22.0,3.0,8.0,13.0,6.0,20.0,24.0,...,17.0,15.0,19.0,48.0,15.0,10.0,12.0,17.0,11.0,2
1230,1230,polyps,27.0,20.0,24.0,9.0,7.0,7.0,15.0,14.0,...,20.0,14.0,22.0,52.0,9.0,26.0,7.0,24.0,6.0,2
1231,1231,polyps,24.0,24.0,26.0,12.0,16.0,11.0,10.0,10.0,...,17.0,18.0,18.0,57.0,16.0,17.0,12.0,11.0,12.0,2


divide by train and test

In [525]:
#randomly take 30 examples of each class to train 
df_train = df.groupby('class').apply(lambda x: x.sample(n=30, random_state=1)).reset_index(drop = True)

#delete df_train from df to get df_test 
df_test = df[~df['example'].isin(df_train['example'])]

#
print(len(df))
print(len(df_train))
print(len(df_test))
print(len(df_train) + len(df_test))

1233
90
1143
1233


In [526]:
# Separate features and labels
X_train = df_train.drop(['example', 'class', 'class_label'], axis=1)
y_train = df_train['class_label'].values

X_test = df_test.drop(['example', 'class', 'class_label'], axis=1)
y_test = df_test['class_label'].values

# Convert labels to integers
y_train = y_train.astype(int)
y_test = y_test.astype(int)


In [527]:
X_test.values

array([[23., 14., 11., ..., 10., 14.,  7.],
       [29., 14., 16., ...,  8., 27.,  2.],
       [11., 24.,  9., ...,  7., 27.,  6.],
       ...,
       [27., 20., 24., ...,  7., 24.,  6.],
       [24., 24., 26., ..., 12., 11., 12.],
       [27., 26.,  6., ...,  9., 18.,  7.]])

In [528]:
import numpy as np
from scipy.spatial.distance import euclidean
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

In [529]:
# Define the k-NN function
def k_nn_predict(test_example, k):
    distances = np.array([euclidean(test_example, train_example) for train_example in X_train.values])
    indices = np.argsort(distances)[:k]
    k_nearest_labels = y_train[indices]
    return np.bincount(k_nearest_labels).argmax()

# Evaluate for k=1, 3, and 5
for k in [1, 3, 5]:
    predictions = [k_nn_predict(test_example, k) for test_example in X_test.values]

    # Confusion Matrix
    cm = confusion_matrix(y_test, predictions)
    print(f"Confusion Matrix (k={k}):\n{cm}")

    # Metrics
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted')
    recall = recall_score(y_test, predictions, average='weighted')
    print(f"Accuracy (k={k}): {accuracy:.4f}")
    print(f"Precision (k={k}): {precision:.4f}")
    print(f"Recall (k={k}): {recall:.4f}")
    print("="*40)

Confusion Matrix (k=1):
[[133   3  10]
 [  0 408   6]
 [ 72  27 484]]
Accuracy (k=1): 0.8968
Precision (k=1): 0.9140
Recall (k=1): 0.8968
Confusion Matrix (k=3):
[[127   3  16]
 [  1 410   3]
 [ 79  13 491]]
Accuracy (k=3): 0.8994
Precision (k=3): 0.9180
Recall (k=3): 0.8994
Confusion Matrix (k=5):
[[125   5  16]
 [  0 410   4]
 [ 85  14 484]]
Accuracy (k=5): 0.8915
Precision (k=5): 0.9120
Recall (k=5): 0.8915


In [530]:
# Define a function to calculate confusion matrix
def calculate_confusion_matrix(actual, predicted):
    classes = np.unique(actual)
    matrix = np.zeros((len(classes), len(classes)))

    for i in range(len(actual)):
        matrix[actual[i], predicted[i]] += 1

    return matrix

# Define a function to calculate accuracy
def calculate_accuracy(actual, predicted):
    correct = sum(np.array(actual) == np.array(predicted))
    total = len(actual)
    return correct / total

# Define a function to calculate precision
def calculate_precision(actual, predicted, class_label):
    true_positive = sum((np.array(actual) == class_label) & (np.array(predicted) == class_label))
    false_positive = sum((np.array(actual) != class_label) & (np.array(predicted) == class_label))

    if true_positive + false_positive == 0:
        return 0
    else:
        return true_positive / (true_positive + false_positive)

# Define a function to calculate recall
def calculate_recall(actual, predicted, class_label):
    true_positive = sum((np.array(actual) == class_label) & (np.array(predicted) == class_label))
    false_negative = sum((np.array(actual) == class_label) & (np.array(predicted) != class_label))

    if true_positive + false_negative == 0:
        return 0
    else:
        return true_positive / (true_positive + false_negative)

# Evaluate for k=1, 3, and 5
for k in [1, 3, 5]:
    predictions = [k_nn_predict(test_example, k) for test_example in X_test.values]

    # Confusion Matrix
    cm = calculate_confusion_matrix(y_test, predictions)
    print(f"Confusion Matrix (k={k}):\n{cm}")

    # Metrics
    accuracy = calculate_accuracy(y_test, predictions)
    precision = np.mean([calculate_precision(y_test, predictions, label) for label in np.unique(y_test)])
    recall = np.mean([calculate_recall(y_test, predictions, label) for label in np.unique(y_test)])
    
    print(f"Accuracy (k={k}): {accuracy:.4f}")
    print(f"Precision (k={k}): {precision:.4f}")
    print(f"Recall (k={k}): {recall:.4f}")
    print("=" * 40)


Confusion Matrix (k=1):
[[133.   3.  10.]
 [  0. 408.   6.]
 [ 72.  27. 484.]]
Accuracy (k=1): 0.8968
Precision (k=1): 0.8494
Recall (k=1): 0.9089
Confusion Matrix (k=3):
[[127.   3.  16.]
 [  1. 410.   3.]
 [ 79.  13. 491.]]
Accuracy (k=3): 0.8994
Precision (k=3): 0.8462
Recall (k=3): 0.9008
Confusion Matrix (k=5):
[[125.   5.  16.]
 [  0. 410.   4.]
 [ 85.  14. 484.]]
Accuracy (k=5): 0.8915
Precision (k=5): 0.8371
Recall (k=5): 0.8922
